In [1]:
import pandas as pd
import zipfile
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
df_feat = pd.read_csv('feat_cup.csv', delimiter=',')
df_objects = pd.read_csv('objects_cup.csv', delimiter=',')
df_excel = pd.read_excel('feat_dict_cup.xlsx')

In [55]:
df_objects.groupby('holder_id')['card_uk'].count().reset_index().sort_values('card_uk')

,holder_id,card_uk
0,0000037dc4be61f378896fdcf92fbdc2149dddce77974f...,1
323344,a30fa54e4c4e4c45301fbc2305ea9e0abd733329b27596...,1
323343,a30f758600fd3ac2a8f68635319c6a36406235c9f7b246...,1
323342,a30f33a0969b2ad87e96f4d8b45da0f48c3fbdc2eb783e...,1
323341,a30f2f297e1f2252357a8da5dbb6c9f060bfa7648fb1e2...,1
...,...,...
300697,97b8f6fcd392c1333ad4d9eaf553bbb08caa86a9406c3c...,22
126697,3fc77308c2aa1c7373fe161ba6f81f826d5b23b76bdb8a...,34
2146,01073424ca6704a4ed626b930e63bc05a12f028e320b0c...,89
500192,fb87393a13e055485cd0b5eb90bdc5f8c4f602bb2799c2...,321


In [59]:
df_objects[df_objects['owner_id'] != df_objects['holder_id']].groupby('owner_id')['holder_id'].count().reset_index().describe()

,holder_id
count,101214.000000
mean,1.263995
std,3.104375
min,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,966.000000


In [40]:
df_objects['cardapp_date'] = pd.to_datetime(df_objects['cardapp_date'])

In [47]:
df_objects['month'] = df_objects['cardapp_date'].dt.month

In [46]:
df_objects.groupby('month')['owner_id'].count().reset_index().sort_values()

,month,owner_id
0,1.0,47503
1,2.0,47474
2,3.0,54283
3,4.0,46178
4,5.0,42180
5,6.0,46799
6,7.0,52236
7,8.0,55771
8,9.0,59730
9,10.0,54101


In [3]:
test_1 = df_objects[df_objects['owner_id'] != df_objects['holder_id']].drop_duplicates(subset =  ['owner_id','holder_id']).\
                                    drop_duplicates('holder_id').groupby('owner_id')['holder_id'].count().reset_index().\
    sort_values('holder_id', ascending = False).reset_index().rename(columns = {'owner_id':'client_id', 'holder_id':'col_holders'})

In [4]:
test_1.drop('index', axis=1, inplace=True)

In [5]:
df_feat = df_feat.merge(test_1, how='left', on='client_id')
df_feat['is_owner'] = df_feat.col_holders.apply(lambda x: 1 if x>0 else 0)
df_feat.drop('col_holders', axis=1, inplace=True)

In [6]:
categorial_features = df_feat.select_dtypes(include=['object']).columns.to_list()

df_feat = df_feat.drop_duplicates(subset='client_id', keep='last')
        
feat_eda = df_feat[categorial_features].drop(columns = (['client_id','dt']))

for i in feat_eda.columns:
    test_city = feat_eda[i].value_counts(normalize = True).reset_index().rename(columns = ({'index': 'city'}))
    city_list = test_city['city'].to_list()
    p_city_list = test_city[i].to_list()
    test_weight_city = pd.Series(np.random.choice(city_list, replace = True, p = p_city_list, size = feat_eda[i].isna().sum()))\
    .value_counts()
    target_weight = np.random.choice(city_list, replace = True, p = p_city_list, size = feat_eda[i].isna().sum()).tolist()
    new_values = pd.Series(target_weight, index=feat_eda.index[feat_eda[i].isna()])
    feat_eda[i].fillna(new_values, inplace=True)
if (len(feat_eda) == len(df_feat)) & (feat_eda.isna().sum().sum() == 0):
    print('Всё в норме, пропусков нет, длина исходного массива совпадает с получившимся')
else:
    print('Что-то пошло не так')

coder_other = []
for i in feat_eda.columns:
    if len(feat_eda[i].value_counts()) > 6:
        coder_other.append(i)
print(f'Претенденты на заполнение other {coder_other}')

feat_eda['country_coder'] = feat_eda['country'].apply(lambda x: 'RUS' if x == 'RUS' else 'OTHER')
feat_eda['clientoutflowstatus_coder'] = feat_eda['clientoutflowstatus'].apply(lambda x: 'ACTIVE' if x == 'ACTIVE' else 'OTHER')
addrref_check = feat_eda['addrref'].value_counts(normalize = True).reset_index()
addrref_small = addrref_check[addrref_check['addrref'] < 0.03]
addrref_small['checker'] = 'OTHER'
end_addrref = addrref_check.merge(addrref_small, on = ['index','addrref'], how = 'left')
end_addrref['checker'] = end_addrref['checker'].fillna(pd.Series(end_addrref['index'].values[:5]))
feat_eda = feat_eda.merge(end_addrref[['index','checker']], left_on = 'addrref', right_on = 'index').drop(columns = 'index')\
.rename(columns = {'checker':'addrref_coder'})
new_df_with_coders = feat_eda.drop(columns = (['country','addrref']))
new_df_with_coders = new_df_with_coders.drop(columns = ['clientoutflowstatus'])
new_df_with_coders = pd.get_dummies(new_df_with_coders)
df_feat_new_with_encoders = df_feat.drop(columns = categorial_features)\
            .merge(new_df_with_coders, left_index = True, right_index = True)
correlations = df_feat_new_with_encoders.corr().unstack().reset_index().rename(columns = {0:'correlations'})


Всё в норме, пропусков нет, длина исходного массива совпадает с получившимся
Претенденты на заполнение other ['country', 'clientoutflowstatus', 'srvpackage', 'addrref']


C:\Users\User\AppData\Local\Temp\ipykernel_14328\383242814.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  addrref_small['checker'] = 'OTHER'


In [7]:
stats_nan = df_feat_new_with_encoders.isna().sum().reset_index().rename(columns = {0:'count_nan'})
stats_nan['all'] = len(df_feat_new_with_encoders)
stats_nan['nan_percent'] = stats_nan['count_nan'] / stats_nan['all']
stats_nan = stats_nan.sort_values('nan_percent', ascending = False)
stats_filter_columns = stats_nan[stats_nan['nan_percent'] <= np.percentile(stats_nan['nan_percent'], 50)]
values_stats_nan = stats_filter_columns['index'].values
df_feat_new_with_encoders_without_high_nan = df_feat_new_with_encoders[values_stats_nan]

In [8]:
columns_to_drop_with_high_cor = correlations[(correlations['correlations'] < 1) & (correlations['correlations'] > 0.7)]\
                                                .drop_duplicates('level_0')['level_0'].values

In [9]:
df_feat_new_with_encoders_without_high_nan = df_feat_new_with_encoders_without_high_nan.drop(columns=[col for col in columns_to_drop_with_high_cor if col in df_feat_new_with_encoders_without_high_nan.columns])

In [10]:
df_feat_new_with_encoders_without_high_nan['is_owner'].value_counts()

0    186799
1     42234
Name: is_owner, dtype: int64

In [11]:
a = df_feat_new_with_encoders_without_high_nan.columns

In [12]:
a

Index(['transaction_category_pharmacy_percent_amt_2m',
       'transaction_category_pharmacy_percent_cnt_2m',
       'transaction_category_pharmacy_sum_cnt_m2',
       'by_category__amount__SUM__eoperation_type_name__perevod_po_nomeru_telefona',
       'transaction_category_fastfood_percent_amt_2m',
       'transaction_category_fastfood_percent_cnt_2m',
       'transaction_category_other_retail_purchase_percent_amt_2m',
       'transaction_category_other_retail_purchase_sum_amt_m2',
       'transaction_category_other_retail_purchase_sum_cnt_m2',
       'transaction_category_other_retail_purchase_percent_cnt_2m',
       'transaction_category_cash_percent_amt_2m',
       'transaction_category_cash_percent_cnt_2m',
       'transaction_category_cash_sum_cnt_m2',
       'hdb_bki_other_active_cc_outstanding',
       'transaction_category_supermarket_percent_amt_2m',
       'transaction_category_supermarket_percent_cnt_2m',
       'hdb_bki_active_cc_max_outstand', 'hdb_bki_total_pil_max_limit

In [13]:
fill0 = [i for i in a if i.startswith('transaction')]

In [14]:
fill0.append('by_category__amount__SUM__eoperation_type_name__perevod_po_nomeru_telefona')

In [15]:
fill0.extend([i for i in a if i.startswith('hdb')])

In [16]:
fill0.extend([i for i in a if i.startswith('avg')])

In [17]:
fill0.append('sal_rur_amt_curr_v2')

In [18]:
fill0.append('accountsalary_flag')

In [19]:
df_feat_new_with_encoders_without_high_nan[fill0] = \
                                df_feat_new_with_encoders_without_high_nan[fill0].fillna(0)

In [20]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
final = imputer.fit_transform(df_feat_new_with_encoders_without_high_nan)

In [21]:
fi = pd.DataFrame(final, columns=a)

In [22]:
fi.is_owner.value_counts()

0.0    186799
1.0     42234
Name: is_owner, dtype: int64

In [23]:
X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

In [36]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

scaler = StandardScaler()

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)



X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_scaled, y_train)

X_train_resampled = X_train_resampled.astype(np.int32)

input_layer = Input(shape=(X_train_resampled.shape[1],))

embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)



hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
dropout1 = Dropout(0.5)(hidden_layer1)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(dropout1)
dropout2 = Dropout(0.5)(hidden_layer2)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dropout2)

output_layer = Dense(1, activation='sigmoid')(hidden_layer3)

model = Model(inputs=input_layer, outputs=output_layer)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model_7/embedding_262/embedding_lookup' defined at (most recent call last):
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\pla

### Без оверсемплинга

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [27]:
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=42)

In [28]:
model = LogisticRegression()

In [29]:
model.fit(X_train, y_train)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [30]:
model.score(X_test, y_test)

0.8312141940434765

In [32]:
pip install catboost


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 73.9/73.9 MB 1.0 MB/s eta 0:00:00


In [33]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier()

catboost.fit(X_train, y_train)

Learning rate set to 0.088378
0:	learn: 0.6182600	total: 253ms	remaining: 4m 12s
1:	learn: 0.5643611	total: 356ms	remaining: 2m 57s
2:	learn: 0.5208892	total: 490ms	remaining: 2m 42s
3:	learn: 0.4860710	total: 592ms	remaining: 2m 27s
4:	learn: 0.4544138	total: 703ms	remaining: 2m 19s
5:	learn: 0.4310502	total: 831ms	remaining: 2m 17s
6:	learn: 0.4139478	total: 941ms	remaining: 2m 13s
7:	learn: 0.3994849	total: 1.06s	remaining: 2m 11s
8:	learn: 0.3888307	total: 1.16s	remaining: 2m 7s
9:	learn: 0.3786900	total: 1.3s	remaining: 2m 8s
10:	learn: 0.3712136	total: 1.41s	remaining: 2m 7s
11:	learn: 0.3653611	total: 1.52s	remaining: 2m 5s
12:	learn: 0.3602452	total: 1.64s	remaining: 2m 4s
13:	learn: 0.3557175	total: 1.79s	remaining: 2m 6s
14:	learn: 0.3511171	total: 1.93s	remaining: 2m 6s
15:	learn: 0.3481943	total: 2.04s	remaining: 2m 5s
16:	learn: 0.3453331	total: 2.18s	remaining: 2m 6s
17:	learn: 0.3433354	total: 2.36s	remaining: 2m 9s
18:	learn: 0.3413195	total: 2.54s	remaining: 2m 11s
19:

In [208]:
X_pred = catboost.predict(X_test)

In [210]:
from sklearn.metrics import classification_report
print(classification_report(X_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.95      0.89      0.92     65666
         1.0       0.49      0.69      0.57      9915

    accuracy                           0.86     75581
   macro avg       0.72      0.79      0.75     75581
weighted avg       0.89      0.86      0.87     75581



### С оверсемплингом

In [44]:
pip install imblearn


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ------------------------------------ 226.0/226.0 kB 726.5 kB/s eta 0:00:00


In [45]:
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE().fit_resample(X, Y)
y_resampled.value_counts()

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

1.0    186799
0.0    186799
Name: is_owner, dtype: int64

In [47]:
model = LogisticRegression()

In [48]:
model.fit(X_train, y_train)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [49]:
model.score(X_test, y_test)

0.7202079683343067

In [112]:
from catboost import CatBoostClassifier

catboost = CatBoostClassifier(l2_leaf_reg=1,
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=True)

catboost.fit(X_train, y_train)

0:	learn: 0.6318289	total: 309ms	remaining: 5m 8s
1:	learn: 0.5849996	total: 595ms	remaining: 4m 56s
2:	learn: 0.5096673	total: 861ms	remaining: 4m 46s
3:	learn: 0.4592881	total: 1.09s	remaining: 4m 32s
4:	learn: 0.4402512	total: 1.35s	remaining: 4m 29s
5:	learn: 0.3946773	total: 1.6s	remaining: 4m 24s
6:	learn: 0.3664434	total: 1.89s	remaining: 4m 28s
7:	learn: 0.3528585	total: 2.22s	remaining: 4m 35s
8:	learn: 0.3437503	total: 2.47s	remaining: 4m 32s
9:	learn: 0.3270515	total: 2.69s	remaining: 4m 25s
10:	learn: 0.3158142	total: 2.92s	remaining: 4m 22s
11:	learn: 0.3112493	total: 3.16s	remaining: 4m 20s
12:	learn: 0.3048823	total: 3.43s	remaining: 4m 20s
13:	learn: 0.2979362	total: 3.63s	remaining: 4m 15s
14:	learn: 0.2952592	total: 3.87s	remaining: 4m 14s
15:	learn: 0.2891351	total: 4.1s	remaining: 4m 12s
16:	learn: 0.2832017	total: 4.34s	remaining: 4m 11s
17:	learn: 0.2786852	total: 4.6s	remaining: 4m 10s
18:	learn: 0.2765943	total: 4.81s	remaining: 4m 8s
19:	learn: 0.2736636	total:

In [113]:
y_pred = catboost.predict(X_test)

In [114]:
from sklearn.metrics import classification_report
print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

         0.0       0.95      0.87      0.91     40638
         1.0       0.86      0.94      0.90     34082

    accuracy                           0.90     74720
   macro avg       0.90      0.91      0.90     74720
weighted avg       0.91      0.90      0.90     74720



In [53]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

In [54]:
model = tf.keras.Sequential([
    layers.Dense(32, activation='relu', input_dim=X_train.shape[1]),
    layers.Dense(16, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)


predictions = model.predict(new_data)

Epoch 1/10
7823/7823 [==============================] - 51s 6ms/step - loss: 3107.2209 - accuracy: 0.6453 - val_loss: 5127.2896 - val_accuracy: 0.6528
Epoch 2/10
7823/7823 [==============================] - 52s 7ms/step - loss: 647.9689 - accuracy: 0.6293 - val_loss: 24.6616 - val_accuracy: 0.5391
Epoch 3/10
7823/7823 [==============================] - 48s 6ms/step - loss: 5.1767 - accuracy: 0.5043 - val_loss: 4.6025 - val_accuracy: 0.4995
Epoch 4/10
7823/7823 [==============================] - 47s 6ms/step - loss: 1.4748 - accuracy: 0.4997 - val_loss: 0.8654 - val_accuracy: 0.5009
Epoch 5/10
7823/7823 [==============================] - 46s 6ms/step - loss: 2.0833 - accuracy: 0.5001 - val_loss: 0.7870 - val_accuracy: 0.4992
Epoch 6/10
7823/7823 [==============================] - 50s 6ms/step - loss: 0.7444 - accuracy: 0.4988 - val_loss: 0.6931 - val_accuracy: 0.4992
Epoch 7/10
7823/7823 [==============================] - 57s 7ms/step - loss: 0.6932 - accuracy: 0.5007 - val_loss: 0.6931

NameError: name 'new_data' is not defined

In [55]:
from sklearn.neural_network import MLPClassifier

In [61]:
pd.Series(y_resampled).value_counts()

1.0    186799
0.0    186799
Name: is_owner, dtype: int64

In [56]:
model = MLPClassifier(hidden_layer_sizes=(50, 30), activation='relu', solver='adam', alpha=0.01)
model.fit(X_train, y_train)

MLPClassifier(alpha=0.01, hidden_layer_sizes=(50, 30))

In [57]:
y_pred = model.predict(X_test)

In [58]:
print(classification_report(y_pred, y_test))

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

         0.0       1.00      0.50      0.67    123288
         1.0       0.00      0.00      0.00         0

    accuracy                           0.50    123288
   macro avg       0.50      0.25      0.33    123288
weighted avg       1.00      0.50      0.67    123288



C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [64]:
from sklearn.utils import class_weight

In [82]:
from tensorflow import keras

In [102]:
X_train

,transaction_category_pharmacy_percent_amt_2m,transaction_category_pharmacy_percent_cnt_2m,transaction_category_pharmacy_sum_cnt_m2,by_category__amount__SUM__eoperation_type_name__perevod_po_nomeru_telefona,transaction_category_fastfood_percent_amt_2m,transaction_category_fastfood_percent_cnt_2m,transaction_category_other_retail_purchase_percent_amt_2m,transaction_category_other_retail_purchase_sum_amt_m2,transaction_category_other_retail_purchase_sum_cnt_m2,transaction_category_other_retail_purchase_percent_cnt_2m,...,srvpackage_Комфорт,srvpackage_КомфортUltra,srvpackage_КомфортUltrafree,srvpackage_Комфортfree,srvpackage_Корпоративный,srvpackage_Максимум+,srvpackage_Максимум+free,srvpackage_МаксимумUltra,srvpackage_МаксимумUltrafree,addrref_coder_г. Санкт - Петербург
349363,0.053145,0.083475,7.428864,3951.870185,0.021537,0.044148,0.004980,295.677264,1.000000,0.011602,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000
337901,0.001017,0.020727,4.145459,2185.634980,0.001862,0.053662,0.017115,18803.516072,6.803643,0.045371,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000
92453,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.00000
83146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.00000
360757,0.000244,0.002400,0.422343,26938.538984,0.004255,0.014398,0.061844,18154.891760,6.645769,0.057453,...,0.0,0.0,0.0,0.0,0.577657,0.0,0.0,0.0,0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,0.003496,0.024595,4.458740,480.734794,0.007151,0.125158,0.022301,16719.756092,5.541260,0.033396,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.45874
365838,0.007848,0.015584,2.072601,15322.140064,0.029020,0.080195,0.063810,24456.559490,11.564393,0.086535,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000
131932,0.006781,0.020833,1.000000,272.727273,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000
146867,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000


In [103]:
class_weights = class_weight.compute_class_weight(class_weight= 'balanced', y = y_train, classes = np.unique(y_train))

model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(86,)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
7823/7823 [==============================] - 48s 6ms/step - loss: 2643.6892 - accuracy: 0.5272
Epoch 2/10
7823/7823 [==============================] - 47s 6ms/step - loss: 5.6764 - accuracy: 0.4990
Epoch 3/10
7823/7823 [==============================] - 46s 6ms/step - loss: 1.1332 - accuracy: 0.5001
Epoch 4/10
7823/7823 [==============================] - 48s 6ms/step - loss: 0.7561 - accuracy: 0.4980
Epoch 5/10
7823/7823 [==============================] - 42s 5ms/step - loss: 0.8899 - accuracy: 0.4997
Epoch 6/10
7823/7823 [==============================] - 46s 6ms/step - loss: 0.7544 - accuracy: 0.5009
Epoch 7/10
7823/7823 [==============================] - 51s 6ms/step - loss: 0.7446 - accuracy: 0.4985
Epoch 8/10
7823/7823 [==============================] - 45s 6ms/step - loss: 0.6940 - accuracy: 0.4999
Epoch 9/10
7823/7823 [==============================] - 41s 5ms/step - loss: 0.6982 - accuracy: 0.5004
Epoch 10/10
7823/7823 [==============================] - 42s 5ms/step 

In [106]:
y_pred = model.predict(X_test)

3853/3853 [==============================] - 16s 4ms/step


In [110]:
pd.Series([y_pred])

0    [[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0...
dtype: object

In [107]:
classification_report(y_pred, y_test)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets

In [132]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

model = Sequential()
model.add(Dense(128, activation='relu', input_dim=86))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])


model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))


loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/50


ValueError: in user code:

    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "C:\Users\User\AppDat

In [144]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from tensorflow.keras.regularizers import l2


X_resampled, y_resampled = SMOTE().fit_resample(X, Y)
y_resampled.value_counts()

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model = Sequential()
model.add(Dense(256, activation='relu', input_dim=86, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))

opt = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

loss, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

Epoch 1/50
9340/9340 [==============================] - 97s 10ms/step - loss: 0.6567 - accuracy: 0.7244 - val_loss: 0.7446 - val_accuracy: 0.4981
Epoch 2/50
9340/9340 [==============================] - 101s 11ms/step - loss: 0.5639 - accuracy: 0.7296 - val_loss: 0.6973 - val_accuracy: 0.5119
Epoch 3/50
9340/9340 [==============================] - 90s 10ms/step - loss: 0.5605 - accuracy: 0.7313 - val_loss: 0.6778 - val_accuracy: 0.6469
Epoch 4/50
9340/9340 [==============================] - 91s 10ms/step - loss: 0.5564 - accuracy: 0.7350 - val_loss: 0.7795 - val_accuracy: 0.5029
Epoch 5/50
9340/9340 [==============================] - 100s 11ms/step - loss: 0.5565 - accuracy: 0.7345 - val_loss: 0.6327 - val_accuracy: 0.6959
Epoch 6/50
9340/9340 [==============================] - 90s 10ms/step - loss: 0.5547 - accuracy: 0.7355 - val_loss: 0.6272 - val_accuracy: 0.6687
Epoch 7/50
9340/9340 [==============================] - 93s 10ms/step - loss: 0.5538 - accuracy: 0.7350 - val_loss: 0.7129

In [145]:
y_pred = model.predict(X_test)

2335/2335 [==============================] - 11s 5ms/step


In [146]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_pred_classes, y_true))

In [118]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=86, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Dense(2, activation='softmax'))


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ---------------------------------------- 1.0/1.0 MB 1.4 MB/s eta 0:00:00


In [119]:
import lightgbm as lgb

In [120]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

In [122]:
params = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1,
    'class_weight': 'balanced'
}

In [123]:
num_round = 1000
model = lgb.train(params, train_data, num_round, valid_sets=[test_data], early_stopping_rounds=20)

C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: class_weight
[1]	valid_0's auc: 0.891451
Training until validation scores don't improve for 20 rounds
[2]	valid_0's auc: 0.903931
[3]	valid_0's auc: 0.911335
[4]	valid_0's auc: 0.914057
[5]	valid_0's auc: 0.914387
[6]	valid_0's auc: 0.915159
[7]	valid_0's auc: 0.916936
[8]	valid_0's auc: 0.917541
[9]	valid_0's auc: 0.918187
[10]	valid_0's auc: 0.918549
[11]	valid_0's auc: 0.919518
[12]	valid_0's auc: 0.919419
[13]	valid_0's auc: 0.920632
[14]	valid_0's auc: 0.921209
[15]	valid_0's auc: 0.922329
[16]	valid_0's auc: 0.92291
[17]	valid_0's auc: 0.923575
[18]	valid_0's auc: 0.924103
[19]	valid_0's auc: 0.924579
[20]	valid_0's auc: 0.927474
[21]	valid_0's auc: 0.928661
[22]	valid_0's auc: 0.929048
[23]	valid_0's auc: 0.929981
[24]	valid_0's auc: 0.931123
[25]	valid_0's auc: 0.932359
[26]	valid_0's auc: 0.932839
[27]	valid_0's auc: 0.93323
[28]	valid_0's auc: 0.933779
[29]	valid_0's auc: 0.934585
[30]	valid_0's auc: 0.935308
[31]	valid_0's auc: 0.93632

In [124]:
model

In [ ]:
from sklea

In [128]:
y_test

94891     0.0
153423    0.0
351509    1.0
84657     1.0
250466    1.0
         ... 
301003    1.0
271062    1.0
124153    0.0
257616    1.0
123250    0.0
Name: is_owner, Length: 74720, dtype: float64

In [127]:
y_pred = model.predict(X_test)
y_pred_class = [1 if pred >= 0.5 else 0 for pred in y_pred]
print(classification_report(y_pred_class, y_test))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91     40796
           1       0.86      0.94      0.90     33924

    accuracy                           0.90     74720
   macro avg       0.90      0.91      0.90     74720
weighted avg       0.91      0.90      0.90     74720



In [157]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, y_train = SMOTE().fit_resample(X_train, y_train)

embedding_dim = 10

input_layer = Input(shape=(X_train.shape[1],))

embedding_layer = Embedding(X_train.shape[1], embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)

hidden_layer = Dense(128, activation='relu')(flatten_layer)

output_layer = Dense(1, activation='sigmoid')(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Обучение модели
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model_2/embedding_2/embedding_lookup' defined at (most recent call last):
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\platf

In [156]:
print(input_layer.shape)

(None, 86)


In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, y_train = SMOTE().fit_resample(X_train, y_train)


from tensorflow.keras.regularizers import l2


input_layer = Input(shape=(X_train_resampled.shape[1],))


embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)


hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)

output_layer = Dense(1, activation='sigmoid')(hidden_layer3)


model = Model(inputs=input_layer, outputs=output_layer)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=16, validation_data=(X_test, y_test))






Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'model_3/embedding_89/embedding_lookup' defined at (most recent call last):
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tornado\plat

In [161]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE

X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']


scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)


X_train_int = X_train_scaled.astype(int)


X_train, X_test, y_train, y_test = train_test_split(X_train_int, y_train, test_size=0.2, random_state=42)


X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)


embedding_dim = 10

input_layer = Input(shape=(X_train_resampled.shape[1],))


embedding_layer = Embedding(X_train_resampled.max()+1, embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)
hidden_layer = Dense(128, activation='relu')(flatten_layer)

output_layer = Dense(1, activation='sigmoid')(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_data=(X_test, y_test))

MemoryError: Unable to allocate 1023. MiB for an array with shape (1122, 119540) and data type float64

In [223]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']


scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)


X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)


X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)

embedding_dim = 10


input_layer = Input(shape=(X_train_resampled.shape[1],))


embedding_layer = Embedding(X_train_resampled.max()+1, embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)


hidden_layer = Dense(64, activation='relu')(flatten_layer)

output_layer = Dense(1, activation='sigmoid')(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=16, validation_data=(X_test, y_test))


TypeError: Dimension value must be integer or None or have an __index__ method, got value '2.0' with type '<class 'numpy.float64'>'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from imblearn.over_sampling import SMOTE

X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)

embedding_dim = 10

input_layer = Input(shape=(X_train_resampled.shape[1],))

embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)

hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)

output_layer = Dense(1, activation='sigmoid')(hidden_layer3)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
3605/7662 [=============>................] - ETA: 36s - loss: 0.7272 - accuracy: 0.5029

In [193]:
from tensorflow.keras.layers import Dropout

hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
dropout1 = Dropout(0.5)(hidden_layer1)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(dropout1)
dropout2 = Dropout(0.5)(hidden_layer2)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(dropout2)

output_layer = Dense(1, activation='sigmoid')(hidden_layer3)

model = Model(inputs=input_layer, outputs=output_layer)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=16, validation_data=(X_test, y_test))

Epoch 1/10
6127/6127 [==============================] - 57s 9ms/step - loss: 0.4791 - accuracy: 0.8380 - val_loss: 0.3815 - val_accuracy: 0.8543
Epoch 2/10
6127/6127 [==============================] - 50s 8ms/step - loss: 0.3964 - accuracy: 0.8455 - val_loss: 0.3681 - val_accuracy: 0.8612
Epoch 3/10
6127/6127 [==============================] - 51s 8ms/step - loss: 0.3842 - accuracy: 0.8483 - val_loss: 0.3848 - val_accuracy: 0.8527
Epoch 4/10
6127/6127 [==============================] - 52s 8ms/step - loss: 0.3795 - accuracy: 0.8492 - val_loss: 0.3550 - val_accuracy: 0.8627
Epoch 5/10
6127/6127 [==============================] - 51s 8ms/step - loss: 0.3779 - accuracy: 0.8490 - val_loss: 0.3521 - val_accuracy: 0.8622
Epoch 6/10
6127/6127 [==============================] - 50s 8ms/step - loss: 0.3769 - accuracy: 0.8494 - val_loss: 0.3559 - val_accuracy: 0.8607
Epoch 7/10
6127/6127 [==============================] - 53s 9ms/step - loss: 0.3768 - accuracy: 0.8485 - val_loss: 0.3527 - val_ac

In [221]:
from tensorflow.keras.regularizers import l2


input_layer = Input(shape=(X_train_resampled.shape[1],))


embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)


hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)

output_layer = Dense(1, activation='sigmoid')(hidden_layer3)


model = Model(inputs=input_layer, outputs=output_layer)


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=16, validation_data=(X_test, y_test))


Epoch 1/10
18682/18682 [==============================] - 186s 10ms/step - loss: 0.6089 - accuracy: 0.7042 - val_loss: 0.6151 - val_accuracy: 0.5776
Epoch 2/10
 1293/18682 [=>............................] - ETA: 2:44 - loss: 0.5921 - accuracy: 0.7098


KeyboardInterrupt



In [194]:
y_pred = model.predict(X_test)

766/766 [==============================] - 4s 5ms/step


In [196]:
y_pred_test = (y_pred > 50/100).astype(int) # преобразование вероятностей в метки классов (0 или 1)
print(i)
print(classification_report(y_test, y_pred_test))

100
              precision    recall  f1-score   support

         0.0       0.79      1.00      0.88     12347
         1.0       0.99      0.73      0.84     12139

    accuracy                           0.86     24486
   macro avg       0.89      0.86      0.86     24486
weighted avg       0.89      0.86      0.86     24486



In [168]:
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test)
print(classification_report(y_pred_classes, y_true))

TypeError: Singleton array 2 cannot be considered a valid collection.

In [199]:
from sklearn.model_selection import StratifiedKFold

X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']


scaler = MinMaxScaler()

X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)

X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_scaled, y_train)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

embedding_dim = 10

k = 5 
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X_train_scaled, y_train):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]

    X_train_resampled_cv, y_train_resampled_cv = SMOTE().fit_resample(X_train_cv, y_train_cv)

    input_layer = Input(shape=(X_train_resampled_cv.shape[1],))

    embedding_layer = Embedding(int(X_train_resampled_cv.max()+1), embedding_dim)(input_layer)
    flatten_layer = Flatten()(embedding_layer)

    hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
    hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
    hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)

    output_layer = Dense(1, activation='sigmoid')(hidden_layer3)

    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_resampled_cv, y_train_resampled_cv, epochs=10, batch_size=16, validation_data=(X_test_cv, y_test_cv))


ValueError: Found input variables with inconsistent numbers of samples: [97942, 78353]

In [200]:
X_train_scaled = scaler.fit_transform(X_train).astype(np.float32)
X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train_scaled, y_train)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for train_index, test_index in skf.split(X_train_resampled, y_train_resampled):
    X_train_fold, X_test_fold = X_train_resampled[train_index], X_train_resampled[test_index]
    y_train_fold, y_test_fold = y_train_resampled[train_index], y_train_resampled[test_index]

    input_layer = Input(shape=(X_train_fold.shape[1],))

    embedding_layer = Embedding(int(X_train_fold.max()+1), embedding_dim)(input_layer)
    flatten_layer = Flatten()(embedding_layer)

    hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
    hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
    hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)

    output_layer = Dense(1, activation='sigmoid')(hidden_layer3)
    
    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=16, validation_data=(X_test_fold, y_test_fold))


Epoch 1/10
3918/3918 [==============================] - 27s 6ms/step - loss: 0.4670 - accuracy: 0.8355 - val_loss: 0.3779 - val_accuracy: 0.8587
Epoch 2/10
3918/3918 [==============================] - 25s 6ms/step - loss: 0.3764 - accuracy: 0.8519 - val_loss: 0.3599 - val_accuracy: 0.8571
Epoch 3/10
3918/3918 [==============================] - 27s 7ms/step - loss: 0.3631 - accuracy: 0.8530 - val_loss: 0.3985 - val_accuracy: 0.8460
Epoch 4/10
3918/3918 [==============================] - 28s 7ms/step - loss: 0.3549 - accuracy: 0.8551 - val_loss: 0.3503 - val_accuracy: 0.8575
Epoch 5/10
3918/3918 [==============================] - 31s 8ms/step - loss: 0.3502 - accuracy: 0.8563 - val_loss: 0.3500 - val_accuracy: 0.8590
Epoch 6/10
3918/3918 [==============================] - 32s 8ms/step - loss: 0.3472 - accuracy: 0.8568 - val_loss: 0.3431 - val_accuracy: 0.8595
Epoch 7/10
3918/3918 [==============================] - 31s 8ms/step - loss: 0.3453 - accuracy: 0.8573 - val_loss: 0.3427 - val_ac


KeyboardInterrupt



In [201]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE


In [ ]:
cross_val_score()

In [205]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score

X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)


embedding_dim = 10
input_layer = Input(shape=(X_train_resampled.shape[1],))
embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)
hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer3)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


scoring = {'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'accuracy': 'accuracy',
           'f1': make_scorer(f1_score),
           'roc_auc': make_scorer(roc_auc_score)}

scores = cross_validate(model, X_train_resampled, y_train_resampled, scoring=scoring, cv=5)

print('Precision:', scores['test_precision'].mean())
print('Recall:', scores['test_recall'].mean())
print('Accuracy:', scores['test_accuracy'].mean())
print('F1:', scores['test_f1'].mean())
print('ROC AUC:', scores['test_roc_auc'].mean())

TypeError: Cannot clone object '<keras.engine.functional.Functional object at 0x000002C523720FD0>' (type <class 'keras.engine.functional.Functional'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

f1_scores = []

for train_index, test_index in skf.split(X_train_scaled, y_train):
    X_train, X_test = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train, y_test = y_train[train_index], y_train[test_index]
    
    X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)

    input_layer = Input(shape=(X_train_fold.shape[1],))

    embedding_layer = Embedding(int(X_train_fold.max()+1), embedding_dim)(input_layer)
    flatten_layer = Flatten()(embedding_layer)

    hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
    hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
    hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)

    output_layer = Dense(1, activation='sigmoid')(hidden_layer3)


    model = Model(inputs=input_layer, outputs=output_layer)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    model.fit(X_train_fold, y_train_fold, epochs=10, batch_size=16, validation_data=(X_test_fold, y_test_fold))
    
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5).astype(int)
    f1 = f1_score(y_test, y_pred)
    f1_scores.append(f1)
    
print("F1-score: {:.2f} (+/- {:.2f})".format(np.mean(f1_scores), np.std(f1_scores)))


In [209]:
!pip install -U scikit-learn


[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [213]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils import all_estimators
from keras.wrappers.scikit_learn import KerasClassifier


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)


embedding_dim = 10
input_layer = Input(shape=(X_train_resampled.shape[1],))
embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)
hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer3)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


keras_model = KerasClassifier(model, epochs=10, batch_size=32, verbose=0)


scoring = {'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'accuracy': 'accuracy',
           'f1': make_scorer(f1_score),
           'roc_auc': make_scorer(roc_auc_score)}

scores = cross_validate(keras_model, X_train_resampled, y_train_resampled, scoring=scoring, cv=5)

print('Precision:', scores['test_precision'].mean())
print('Recall:', scores['test_recall'].mean())
print('Accuracy:', scores['test_accuracy'].mean())
print('F1:', scores['test_f1'].mean())
print('ROC AUC:', scores['test_roc_auc'].mean())


C:\Users\User\AppData\Local\Temp\ipykernel_2576\2431171484.py:41: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_model = KerasClassifier(model, epochs=10, batch_size=32, verbose=0)


Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\embedding
......vars
.........0
...layers\flatten
......vars
...layers\input_layer
......vars
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-02 15:50:25         3501
metadata.json                                  2023-04-02 15:50:25           64
variables.h5                                   2023-04-02 15:50:25       504160
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-04-02 15:50:24         3501
metadata.json                                  2

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\wrappers\scikit_learn.py", line 248, in fit
    return super().fit(x, y, **kwargs)
  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\wrappers\scikit_learn.py", line 160, in fit
    self.model = self.build_fn(
  File "C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 7

In [217]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)


X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)


embedding_dim = 10
input_layer = Input(shape=(X_train_resampled.shape[1],))
embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)
hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer3)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

keras_model = KerasClassifier(model, epochs=10, batch_size=32, verbose=0)

scoring = {'precision': make_scorer(precision_score),
'recall': make_scorer(recall_score),
'accuracy': 'accuracy',
'f1': make_scorer(f1_score),
'roc_auc': make_scorer(roc_auc_score)}

scores = cross_validate(keras_model, X_train_resampled, y_train_resampled, scoring=scoring, cv=5, error_score='raise')

print('Precision:', scores['test_precision'].mean())
print('Recall:', scores['test_recall'].mean())
print('Accuracy:', scores['test_accuracy'].mean())
print('F1:', scores['test_f1'].mean())
print('ROC AUC:', scores['test_roc_auc'].mean())

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\dense
......vars
.........0
.........1
...layers\dense_1
......vars
.........0
.........1
...layers\dense_2
......vars
.........0
.........1
...layers\dense_3
......vars
.........0
.........1
...layers\embedding
......vars
.........0
...layers\flatten
......vars
...layers\input_layer
......vars
...optimizer
......vars
.........0
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-04-02 16:02:30         3501
metadata.json                                  2023-04-02 16:02:30           64
variables.h5                                   2023-04-02 16:02:30       504160
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2023-04-02 16:02:30         3501
metadata.json                                  2

ValueError: The first argument to `Layer.call` must always be passed.

In [216]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifierWrapper
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)


X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)


embedding_dim = 10
input_layer = Input(shape=(X_train_resampled.shape[1],))
embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
flatten_layer = Flatten()(embedding_layer)
hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)
output_layer = Dense(1, activation='sigmoid')(hidden_layer3)
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


keras_model = KerasClassifierWrapper(build_fn=model, epochs=10, batch_size=32, verbose=0)

scoring = {'precision': make_scorer(precision_score),
'recall': make_scorer(recall_score),
'accuracy': 'accuracy',
'f1': make_scorer(f1_score),
'roc_auc': make_scorer(roc_auc_score)}

scores = cross_validate(keras_model, X_train_resampled, y_train_resampled, scoring=scoring, cv=5, error_score='raise', n_jobs=-1)

print('Precision:', scores['test_precision'].mean())
print('Recall:', scores['test_recall'].mean())
print('Accuracy:', scores['test_accuracy'].mean())
print('F1:', scores['test_f1'].mean())
print('ROC AUC:', scores['test_roc_auc'].mean())

ImportError: cannot import name 'KerasClassifierWrapper' from 'tensorflow.keras.wrappers.scikit_learn' (C:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\api\_v2\keras\wrappers\scikit_learn\__init__.py)

In [220]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Embedding, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, roc_auc_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

X = fi.drop('is_owner', axis=1)
Y = fi['is_owner']

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X).astype(np.float32)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)


X_train_resampled, y_train_resampled = SMOTE().fit_resample(X_train, y_train)

def create_model():
    embedding_dim = 10
    input_layer = Input(shape=(X_train_resampled.shape[1],))
    embedding_layer = Embedding(int(X_train_resampled.max()+1), embedding_dim)(input_layer)
    flatten_layer = Flatten()(embedding_layer)
    hidden_layer1 = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(flatten_layer)
    hidden_layer2 = Dense(64, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer1)
    hidden_layer3 = Dense(32, activation='relu', kernel_regularizer=l2(0.01))(hidden_layer2)
    output_layer = Dense(1, activation='sigmoid')(hidden_layer3)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

keras_model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)

scoring = {'precision': make_scorer(precision_score),
'recall': make_scorer(recall_score),
'accuracy': 'accuracy',
'f1': make_scorer(f1_score)}

scores = cross_validate(keras_model, X_train_resampled, y_train_resampled, scoring=scoring, cv=5, error_score='raise', n_jobs=-1)

print('Precision:', scores['test_precision'].mean())
print('Recall:', scores['test_recall'].mean())
print('Accuracy:', scores['test_accuracy'].mean())
print('F1:', scores['test_f1'].mean())

Precision: 0.5705509464017341
Recall: 0.8790823711026183
Accuracy: 0.6893905417873263
F1: 0.6316051959503725
